In [1]:
########################################################################################################################
#                                                          IMPORT                                                      #
########################################################################################################################
import torch
import sys
import os
import json
import time
import numpy as np
import argparse
sys.path.append("/home/yifan/projects/cophi/ContraVis")

from umap.umap_ import find_ab_params
from contrast.transfomration import *

from singleVis.SingleVisualizationModel import VisModel
from singleVis.losses import UmapLoss, ReconstructionLoss, SingleVisLoss

from singleVis.data import NormalDataProvider


from singleVis.projector import DVIProjector,ContraProjector
from singleVis.utils import find_neighbor_preserving_rate

########################################################################################################################
#                                                      PARAMETERS                                                   #
########################################################################################################################
"""This serve as an example of DeepVisualInsight implementation in pytorch."""
VIS_METHOD = "DVI" # DeepVisualInsight


TAR_PATH = "/home/yifan/experiments/backdoor/resnet18_CIFAR10/experiment10"
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"

########################################################################################################################
#                                                     LOAD PARAMETERS                                                  #
########################################################################################################################


CONTENT_PATH = REF_PATH
sys.path.append(CONTENT_PATH)
with open(os.path.join(CONTENT_PATH, "config.json"), "r") as f:
    config = json.load(f)
config = config[VIS_METHOD]

# record output information
# now = time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime(time.time())) 
# sys.stdout = open(os.path.join(CONTENT_PATH, now+".txt"), "w")

SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
PREPROCESS = config["VISUALIZATION"]["PREPROCESS"]
GPU_ID = config["GPU"]
GPU_ID = 1
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]

EPOCH_START = 200
EPOCH_END = 200
EPOCH_PERIOD = 1

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
LAMBDA1 = VISUALIZATION_PARAMETER["LAMBDA1"]
LAMBDA2 = VISUALIZATION_PARAMETER["LAMBDA2"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]


S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

VIS_MODEL_NAME = 'Contravis_backdoor' ### saved_as 
EVALUATION_NAME = VISUALIZATION_PARAMETER["EVALUATION_NAME"]

# Define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))

########################################################################################################################
#                                                    TRAINING SETTING                                                  #
########################################################################################################################
# Define data_provider
#TODO
TAE_NET = "resnet18"
tar_net = eval("subject_model.{}()".format(TAE_NET)) 

data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, device=DEVICE, epoch_name='Epoch',classes=CLASSES,verbose=1)
tar_data_provider = NormalDataProvider(TAR_PATH, tar_net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, device=DEVICE, epoch_name='Epoch',classes=CLASSES,verbose=1)


# Define visualization models
model = VisModel(ENCODER_DIMS, DECODER_DIMS)


# Define Losses
negative_sample_rate = 5
min_dist = .1
_a, _b = find_ab_params(1.0, min_dist)
umap_loss_fn = UmapLoss(negative_sample_rate, DEVICE, _a, _b, repulsion_strength=1.0)
recon_loss_fn = ReconstructionLoss(beta=1.0)
single_loss_fn = SingleVisLoss(umap_loss_fn, recon_loss_fn, lambd=LAMBDA1)
# Define Projector
projector = ContraProjector(vis_model=model, content_path=os.path.join(REF_PATH, 'Model', 'Epoch_{}//Contravis.pth'.format(EPOCH_START)), vis_model_name=VIS_MODEL_NAME, device=DEVICE)



/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-31 16:32:07.745296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 16:32:08.406167: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-31 16:32:08.406217: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinf

Finish initialization...
Finish initialization...


In [ ]:
from semantic_distance_calculator import SemanticDistanceCalculator

ss = SemanticDistanceCalculator(data_provider,tar_data_provider,200,200,15)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import functools

@functools.lru_cache(maxsize=None)
def cached_similarity(r_index, t_index):
    return ss.tar_ref_train_data_semantic_similairty_(r_index, t_index)

def find_most_similar_ref_for_each_tar_parallel(num=50):
    max_similarity_indices = np.zeros(num, dtype=int)
    max_similarity_scores = np.zeros(num)
    
    def process_index(t_index):
        highest_similarity = -np.inf
        highest_similarity_r_index = -1
        for r_index in range(num):
            similarity, _, _ = cached_similarity(r_index, t_index)
            if similarity > highest_similarity:
                highest_similarity = similarity
                highest_similarity_r_index = r_index
        return t_index, highest_similarity_r_index, highest_similarity

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_index, range(num)))

    for t_index, r_index, score in results:
        max_similarity_indices[t_index] = r_index
        max_similarity_scores[t_index] = score

    return max_similarity_indices, max_similarity_scores


In [ ]:
max_similarity_indices, max_similarity_scores = find_most_similar_ref_for_each_tar_parallel(num=1000)

In [ ]:
a, b = find_most_similar_ref_for_each_tar_parallel(500)

In [ ]:
import numpy as np

def find_most_similar_ref_for_each_tar(num=50):
    max_similarity_indices = np.zeros(num, dtype=int)  # save the most similar indices
    max_similarity_scores = np.zeros(num)  # save the highest score

    for t_index in range(num):
        highest_similarity = -np.inf  # 初始化为负无穷大
        highest_similarity_r_index = -1  # 初始化索引

        for r_index in range(num):
            # calculate the score
            similarity, _, _ = ss.tar_ref_train_data_semantic_similairty_(r_index, t_index)

            # 检查是否为最高分数
            if similarity > highest_similarity:
                highest_similarity = similarity
                highest_similarity_r_index = r_index

        max_similarity_indices[t_index] = highest_similarity_r_index
        max_similarity_scores[t_index] = highest_similarity

        # 可以在这里打印进度
        if t_index % 1000 == 0:
            print(f"Processed {t_index} target points...")

In [ ]:
ss.tar_ref_train_data_semantic_similairty_(1,2)

In [ ]:
pred1 = data_provider.get_pred(200, data_provider.train_representation(200))
print(pred1[0])

In [ ]:
train_data = tar_data_provider.train_representation(200)
train_data = train_data.reshape(len(train_data), -1)
pred2 = tar_data_provider.get_pred(200, train_data)
print(pred2[0])

In [ ]:

def softmax(x):
    e_x = np.exp(x - np.max(x))  # cal_culate
    return e_x / e_x.sum(axis=0)

In [ ]:
def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
# softmax_vec1 = softmax(pred1[10])
# softmax_vec2 = softmax(pred2[10])
softmax_vec1 = pred1[0]
softmax_vec2 = pred2[0]
cos_sim = cosine_similarity(softmax_vec1, softmax_vec2)
print("Cosine Similarity: ", cos_sim)